## CS431/631 Big Data Infrastructure
### Winter 2019 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Rongrong Su_
* **ID:** _20751802_

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://roegiest.com/bigdata-2019w/assignment1-431.html).

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located.   You will be using the Spark installation in the CS451 course account.   The code in the following cell tells Python how to find this Spark installation.   Before going on, execute that code (by selecting the cell and hitting 'return' while holding down the shift key).   It will take a few seconds to run, and will produce no output.

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [2]:
sc = SparkContext(appName="YourTest", master="local[2]")

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [3]:
import random

num_samples = 100000000

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.1418744


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

*If a circle of radius R is inscribed inside a square with side length 2R, then the area of the circle will be pi*R^2 and the area of the square will be (2R)^2. So the ratio of the area of the circle to the area of the square will be pi/4.

This means that, if you pick N points at random inside the square, approximately N*pi/4 of those points should fall inside the circle. So pi is equal to 4*count/num_samples.

The spark job is counting the number of points inside the circle. The parallelize function is to create distributed dataset which can be operated on in paralle. Filter function is to find the return value of True of inside function. And count function is to get the number of points inside the circle. *

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [5]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
    # your solution to Question 2 here
    sc = SparkContext(appName="YourTest", master="local[2]")
    distFile=sc.textFile('Shakespeare.txt')
    words=distFile.flatMap(simple_tokenize)
    num=words.distinct().collect()
    dn=len(num)
    print('the number of distinct tokens is:',dn)

---
### Important

###### If you fail to run the code please delete line: 'sc = SparkContext(appName="YourTest", master="local[2]")'

In [54]:
count_distinct_tokens()

the number of distinct tokens is: 25975


---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [63]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    # your solution to Question 3 here
    sc = SparkContext(appName="YourTest", master="local[2]")
    distFile=sc.textFile('Shakespeare.txt')
    line=distFile.map(simple_tokenize)
    line2=line.filter(lambda x:x!=[])
    
    def get_pair(t):
        c=[]
        t=list(t)
        tt=set(t)
        ttt=list(tt)
        c=[(x,y) for x in ttt for y in ttt if x!=y]
        return c  

    pair=line2.flatMap(get_pair)
    pair2=pair.distinct().collect()
    num_pair=len(pair2)
    
    return num_pair

---
### Important

###### If you fail to run the code please delete line: 'sc = SparkContext(appName="YourTest", master="local[2]")'

In [64]:
count_distinct_pairs()


1969760

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your code should report (print) the 50 highest-probability tokens, and their probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [62]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
# your solution to Question 4 here 
def top_50_tokens_probabilities():
    sc = SparkContext(appName="YourTest", master="local[2]")
    distFile=sc.textFile('Shakespeare.txt')
    num_lines=distFile.count()
    words=distFile.flatMap(simple_tokenize)
    count_words = words.map(lambda w: (w,1)).reduceByKey(lambda f1,f2: f1+f2)
    
    def cal_p(it):
        p=it[1]/num_lines
        l=(it[0],it[1],p)
        return l
    
    pp=count_words.map(cal_p)
    
    top50=pp.top(50, lambda x:x[1])
    return top50
    # your solution to Question 4 here    


---
### Important

###### If you fail to run the code please delete line: 'sc = SparkContext(appName="YourTest", master="local[2]")'

In [63]:
top_50_tokens_probabilities()

[('the', 27378, 0.22357053030426757),
 ('and', 26082, 0.2129873099348348),
 ('i', 20717, 0.16917637067402702),
 ('to', 19661, 0.16055300592856325),
 ('of', 17473, 0.14268565549004555),
 ('a', 14723, 0.12022897646540039),
 ('you', 13630, 0.11130346731124141),
 ('my', 12490, 0.10199415309738849),
 ('in', 10996, 0.08979405183818126),
 ('that', 10915, 0.0891326005650917),
 ('is', 9137, 0.07461333681752111),
 ('not', 8512, 0.06950954613010175),
 ('with', 7778, 0.06351565434679646),
 ('me', 7777, 0.06350748828169658),
 ('it', 7692, 0.06281337274820754),
 ('for', 7578, 0.06188244132682226),
 ('be', 6867, 0.05607636904081399),
 ('his', 6859, 0.05601104052001502),
 ('your', 6657, 0.05436149536984109),
 ('this', 6606, 0.05394502604974767),
 ('but', 6277, 0.05125839063189012),
 ('he', 6260, 0.05111956752519231),
 ('have', 5885, 0.04805729311274069),
 ('as', 5744, 0.046905877933658885),
 ('thou', 5491, 0.04483986346339153),
 ('him', 5205, 0.04250436884482843),
 ('so', 5056, 0.04128762514494765),
 

In [ ]:
words=distFile.flatMap(simple_tokenize)


---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also report PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [2]:
from simple_tokenize import simple_tokenize
import math
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
    sc = SparkContext(appName="YourTest", master="local[2]")
    distFile=sc.textFile('Shakespeare.txt')
    
    #generate pairs
    line=distFile.map(simple_tokenize)
    line2=line.filter(lambda x:x!=[])
    
    def get_pair(t):
        c=[]
        t=list(t)
        tt=set(t)
        ttt=list(tt)
        c=[(x,y) for x in ttt for y in ttt if x!=y]
        return c  

    pair=line2.flatMap(get_pair)
    pair2=pair.map(lambda w:(w,1)).reduceByKey(lambda f1,f2:f1+f2)
    pair_th=pair2.filter(lambda x:x[1] >=threshold)
    
    #count number of lines
    num_lines=distFile.count()
    
    #count single words
    words=distFile.flatMap(simple_tokenize)
    count_words = words.map(lambda w: (w,1)).reduceByKey(lambda f1,f2: f1+f2)
    c1=count_words.collectAsMap()
    
    #calculate PMI
    def cal_PMI(t):
    
        x = t[0][0]
        y = t[0][1]
        vxy = t[1]
        pxy = t[1]/num_lines
        v1=c1[x]
        px=v1/num_lines
        v2=c1[y]
        py=v2/num_lines
        pmi=math.log10(pxy/(px*py))
        return t[0],pmi,vxy,v1,v2
    
    PMI_value=pair_th.map(cal_PMI).collect()
    
    return PMI_value


---
### Important

###### If you fail to run the code please delete line: 'sc = SparkContext(appName="YourTest", master="local[2]")'

In [3]:
PMI(12)

[(('by', 'william'), 1.013035763669873, 39, 3768, 123),
 (('william', 'by'), 1.013035763669873, 39, 123, 3768),
 (('that', 'might'), 0.4650714559675051, 129, 10915, 496),
 (('might', 'that'), 0.4650714559675051, 129, 496, 10915),
 (('by', 'should'), 0.08599955710162915, 59, 3768, 1573),
 (('by', 'the'), 0.05174076135376003, 949, 3768, 27378),
 (('by', 'time'), 0.07193659374573425, 39, 3768, 1074),
 (('as', 'but'), 0.03344772230308959, 318, 5744, 6277),
 (('should', 'by'), 0.08599955710162915, 59, 1573, 3768),
 (('should', 'the'), -0.1447427387944356, 252, 1573, 27378),
 (('but', 'as'), 0.03344772230308959, 318, 6277, 5744),
 (('the', 'by'), 0.05174076135376003, 949, 27378, 3768),
 (('the', 'should'), -0.1447427387944356, 252, 27378, 1573),
 (('the', 'time'), 0.2005061373593896, 381, 27378, 1074),
 (('time', 'by'), 0.07193659374573425, 39, 1074, 3768),
 (('time', 'the'), 0.2005061373593896, 381, 1074, 27378),
 (('his', 'bear'), 0.02207650807618375, 32, 6859, 543),
 (('his', 'heir'), 0.4

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```


In [3]:
from simple_tokenize import simple_tokenize
from math import log
import math
import random
# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
    # your solution to Question 6 here
    sc = SparkContext(appName="YourTest", master="local[2]")
    distFile=sc.textFile('Shakespeare.txt')

    #get dictinct words
    words=distFile.flatMap(simple_tokenize)
    count_words = words.map(lambda w: (w,1)).reduceByKey(lambda f1,f2: f1+f2)
    dist_words=words.distinct().collect()
    words_list=count_words.collect()
    words_dict=count_words.collectAsMap()

    #get pairs with threshold
    line=distFile.map(simple_tokenize)
    line2=line.filter(lambda x:x!=[])

    def get_pair(t):
        c=[]
        t=list(t)
        tt=set(t)
        ttt=list(tt)
        c=[(x,y) for x in ttt for y in ttt if x!=y]
        return c  

    num_lines=distFile.count()
    pair=line2.flatMap(get_pair)
    pair2=pair.map(lambda w:(w,1)).reduceByKey(lambda f1,f2:f1+f2)
    pair_list=pair2.collect()
    pair_dict=pair2.collectAsMap()

    #calculate PMI
    def cal_PMI(t):
            x = t[0][0]
            y = t[0][1]
            vxy = t[1]
            pxy = t[1]/num_lines
            v1=words_dict[x]
            px=v1/num_lines
            v2=words_dict[y]
            py=v2/num_lines
            pmi=math.log10(pxy/(px*py))
            return t[0],pmi,vxy,v1,v2

    #get ramdom 
    ram=random.sample(dist_words,samp_size)

    #calculate results
    def cal_p(t):
        aa2=pair2.filter(lambda x:x[0][0] == t)
        aa3=aa2.filter(lambda x:x[1] >= threshold)
        PMI_value=aa3.map(cal_PMI).collect()
        return t,PMI_value
    
    result_list=[]
    for i in range(len(ram)):
        a=cal_p(ram[i])
        result_list.append(a)
        
    return result_list


In [5]:
PMI_one_token(2,2)

[("downfall'n", []),
 ("temper'd",
  [(("temper'd", 'thy'), 0.7041000179432212, 2, 12, 4034),
   (("temper'd", 'of'), 0.2435596939642112, 3, 12, 17473),
   (("temper'd", 'when'), 1.2986885510441704, 4, 12, 2052),
   (("temper'd", 'are'), 0.7777187955711639, 2, 12, 3405),
   (("temper'd", 'ill'), 1.8642317085463704, 2, 12, 279),
   (("temper'd", 'was'), 0.9615310487718072, 2, 12, 2230),
   (("temper'd", 'i'), -0.006490954225451103, 2, 12, 20717),
   (("temper'd", 'as'), 0.5506214805857239, 2, 12, 5744),
   (("temper'd", 'so'), 0.7821201056013206, 3, 12, 5056),
   (("temper'd", 'to'), 0.016230308627149692, 2, 12, 19661),
   (("temper'd", 'the'), -0.1275658073363366, 2, 12, 27378),
   (("temper'd", 'better'), 1.5360494668387743, 2, 12, 594),
   (("temper'd", 'that'), 0.2718121717748099, 2, 12, 10915),
   (("temper'd", 'and'), 0.06958628030116855, 3, 12, 26082),
   (("temper'd", 'your'), 0.4865573548682971, 2, 12, 6657)])]

---
### Important

###### If you fail to run the code please delete line: 'sc = SparkContext(appName="YourTest", master="local[2]")'

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.